In [ ]:
# 1. Install & import

In [27]:
import fitz                    
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
# 2. Load PDF

In [4]:
doc = fitz.open("attention_is_all_you_need.pdf")
raw_text = "\n\n".join(page.get_text("text") for page in doc)

In [15]:
raw_text[174:199]

'Attention Is All You Need'

In [ ]:
# 3. Chunk the text

In [19]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

chunks = splitter.split_text(raw_text)
print(f"Generated {len(chunks)} chunks.")

Generated 52 chunks.


In [ ]:
# 4. (Optional) Embed & store in Pinecone

In [30]:
# 5. Flashcard‐generation prompt
client = OpenAI()

level = [beginner, intermediate, advanced, expert]


prompt = f"""
You are a data‐science tutor generating flashcards for a {level[2]}.
Context:
\"\"\"
{chunks[0:5]}
\"\"\"
Output JSON array of objects with "question" and "answer" fields.
"""

response = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "developer",
            "content": "You are an expert tutor who know exactly what students need to learn."},
        {"role": "user",
         "content": prompt}],
    temperature=0.2,
)

cards = response.output_text

In [31]:
cards

'[\n  {\n    "question": "What is the main innovation introduced by the Transformer model?",\n    "answer": "The Transformer model is based solely on attention mechanisms, completely dispensing with recurrence and convolutions."\n  },\n  {\n    "question": "Why is the Transformer model considered more efficient than previous models?",\n    "answer": "The Transformer is more parallelizable and requires significantly less time to train compared to models based on recurrent or convolutional neural networks."\n  },\n  {\n    "question": "What tasks did the Transformer model outperform previous models on?",\n    "answer": "The Transformer outperformed previous models on machine translation tasks, specifically on the WMT 2014 English-to-German and English-to-French translation tasks."\n  },\n  {\n    "question": "What BLEU scores did the Transformer achieve on the WMT 2014 translation tasks?",\n    "answer": "The Transformer achieved a BLEU score of 28.4 on English-to-German and 41.8 on Engl

In [32]:
len(cards)

2416